In [1]:
import torch
import numpy as np
from datetime import datetime, timedelta, date

# use gpu to increase performance
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [2]:
import hjm_ln
from swing_volume_grid import*
from parametric_strategies import nn_strat, pv_strat

# diffusion model configs

In [3]:
f0 = 20.0
ref_date = datetime(2022, 9, 30)  #datetime(2021, 12, 31)
contract_start = datetime(2022, 10, 1) #datetime(2022, 1, 1)
contract_end = datetime(2022, 10, 31) #datetime(2022, 12, 31)
n_ex_dates = (contract_end - contract_start).days + 1
ex_dates = np.array([contract_start + timedelta(days = i) for i in range(n_ex_dates)])

In [4]:
dimension = 1
alpha = 4.0 * torch.ones(dimension, device = device)
sigma = 0.7 * torch.ones(dimension, device = device)
hjm_model = hjm_ln.One_Factor(f0, alpha, sigma, ref_date, ex_dates)

# ----> example for multi dim
#corr_mat = torch.eye(dimension, device = device)
#hjm_model = hjm_ln.Multi_Factor(f0, alpha, sigma, corr_mat, ref_date, ex_dates) # multi_dim (dimension > 1)

# swing configs

In [5]:
Q_min = 140.0 # 1300.0
Q_max = 200.0 # 1900.0
q_min = 0.0 * torch.ones(len(ex_dates))
q_max = 6.0 * torch.ones(len(ex_dates))
grid = Volume_Grid(Q_min, Q_max, q_min, q_max, ex_dates)
strike_price = 20.0

# optimization configs

In [6]:
#optim_infos_dict = {"algo_name" : "Adam", "nb_batches" : 1, "batch_size" : 16384, "n_iterations" : 3000,
                    #"update_infos" : {"lr" : 1e-1, "amsgrad" : True},
                    #"transfer_learning" : {"activate" : False, "n_iter_pre_training" : 300} 
                   #}

In [7]:
optim_infos_dict = {"algo_name" : "pSGLD", "nb_batches" : 1, "batch_size" : 16384, "n_iterations" : 1000,
                    "update_infos" : {"lr" : 1e-1, "sigma": 1e-6, "beta": 0.8, "lambda": 1e-10},
                    "transfer_learning" : {"activate" : False, "n_iter_pre_training" : 300} 
                   }

# pv_strat

In [8]:
pv_strat.Compute_Swing_Price(hjm_model, optim_infos_dict, grid, strike_price, 50, 100000)

--- Training time (pv_strat): 39.7 seconds ---


{'swing_price': 64.97,
 'var_price': 117046.57,
 'swing_price_bang_bang': 64.99,
 'var_price_bang_bang': 116982.23}

# nn_strat

In [9]:
nn_hidden_sizes = [30, 30] # 2 hidden layers with 30 units

In [10]:
nn_strat.Compute_Swing_Price(hjm_model, nn_hidden_sizes, optim_infos_dict, grid, strike_price, 50, 100000)

--- Training time (nn_strat): 484.08 seconds ---


{'swing_price': 65.26,
 'var_price': 117130.58,
 'swing_price_bang_bang': 65.23,
 'var_price_bang_bang': 117128.3}

# transfer learning

In [ ]:
optim_infos_dict = {"algo_name" : "pSGLD", "nb_batches" : 1, "batch_size" : 16384, "n_iterations" : 500,
                    "update_infos" : {"lr" : 1e-1, "sigma": 1e-6, "beta": 0.8, "lambda": 1e-10},
                    "transfer_learning" : {"activate" : True, "n_iter_pre_training" : 1000} 
                   }

In [ ]:
pv_strat.Compute_Swing_Price(hjm_model, optim_infos_dict, grid, strike_price, 50, 100000)

In [ ]:
nn_strat.Compute_Swing_Price(hjm_model, nn_hidden_sizes, optim_infos_dict, grid, strike_price, 50, 100000)